In [261]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                   'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                   'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [262]:
#Some values in the the FlightNumber column are missing.These numbers are meant to increase by 10 with each row
#so 10055 and 10075 need to be put in place.Fill in these missing numbers 
#and make the column an integer column(instead of a float column).

#Find the missing Values in the Column
missing = np.where(df["FlightNumber"].isnull() == True)

#Iterate thru those missing values and then update accordingly
for lst in missing:
    for idx in lst:
        df.iloc[idx,1] = df.iloc[idx-1,1] + 10

#Convert the column from float to int
df["FlightNumber"] = df["FlightNumber"].astype(int)

#Print the Data Frame
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [263]:
#The From_To column would be better as two separate columns! Split each string on the underscore(_) delimiter 
#to give a new temporary DataFrame with the correct values.Assign the correct column names to this temporary DataFrame.
temp_df = df.From_To.str.split('_',expand = True)
temp_df

,0,1
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [264]:
#Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame.
#Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)
temp_df.rename(columns={0:"From_City",1:"To_City"},inplace=True)
temp_df["From_City"] = temp_df.From_City.str.capitalize()
temp_df["To_City"] = temp_df.To_City.str.capitalize()
temp_df

,From_City,To_City
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


In [265]:
#Delete the From_To column from df and attach the temporary DataFrame from the previous questions.
df = pd.concat((df,temp_df),axis=1)
df.drop(columns='From_To',inplace=True)
df

,FlightNumber,RecentDelays,Airline,From_City,To_City
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


In [266]:
#In the RecentDelays column, the values have been entered into the DataFrame as a list.
#We would like each first value in its own column, each second value in its own column, and so on.
#If there isn't an Nth value, the value should be NaN.

#Expand the Series of lists into a DataFrame named delays, rename the columns
#delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
#with delays

max_lst_lngth = df.RecentDelays.map(len).max()
cntr = 0

#First way of creating a DataFrame from Series
for series_idx in df.RecentDelays.str:
    i = list(series_idx)
    if cntr < 1:
        delays_df = pd.DataFrame({'delay_'+str(cntr+1) : i})
    else:
        temp = pd.DataFrame({'delay_'+str(cntr+1) : i})
        delays_df = pd.concat((delays_df,temp),axis=1)
    cntr = cntr + 1

delays_df

#Second method for creating a Dataframe from Series
cntr = 0
for series_idx in df.RecentDelays.str:
    if cntr < 1:
        delays = series_idx.to_frame(name='delay_'+str(cntr+1))
    else:
        temp = series_idx.to_frame(name='delay_'+str(cntr+1))
        delays = pd.concat((delays,temp),axis=1)
    cntr = cntr + 1

delays
df.drop(columns='RecentDelays',inplace=True) 
df = pd.concat((df,delays),axis=1)
df

,FlightNumber,Airline,From_City,To_City,delay_1,delay_2,delay_3
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN
